<a href="https://colab.research.google.com/github/jamesdietle/2022CodingChallenges/blob/main/LogParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are going to Parse some logs!

Refs:
Lets use https://cyberdefenders.org/blueteam-ctf-challenges/42

https://medium.com/mii-cybersec/log-analysis-in-dfir-using-jupyter-notebook-7bb8ea3c015b

Things to do:
- Finish questions
- IP to Geolocation/EDA

In [14]:
!ls

c26-Hammered.zip  file	Hammered  sample_data


In [2]:
!curl https://cyberdefenders.org/blueteam-ctf-challenges/enroll/42 --output c26.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [12]:
#Unzip with the included password
!unzip -P cyberdefenders.org c26-Hammered.zip

Archive:  c26-Hammered.zip
  inflating: Hammered/apache2/www-access.log  
  inflating: Hammered/apache2/www-error.log  
  inflating: Hammered/apache2/www-media.log  
  inflating: Hammered/apt/term.log   
  inflating: Hammered/auth.log       
  inflating: Hammered/daemon.log     
  inflating: Hammered/debug          
  inflating: Hammered/dmesg          
  inflating: Hammered/dmesg.0        
  inflating: Hammered/dpkg.log       
  inflating: Hammered/fontconfig.log  
  inflating: Hammered/fsck/checkfs   
  inflating: Hammered/fsck/checkroot  
  inflating: Hammered/kern.log       
  inflating: Hammered/messages       
 extracting: Hammered/secure         
  inflating: Hammered/udev           
  inflating: Hammered/user.log       


In [15]:
# Did everything unzip correctly?
!ls Hammered/

In [16]:
from collections import deque

In [21]:
findStr= 'WARNING'

with open("Hammered/daemon.log") as f:
  for line in deque(f, maxlen = 10000000):
    if(findStr in line):
      daemon+= [line.strip()]

In [25]:
def search(logfile, findStr):
  daemon= []
  with open(logfile) as f:
    for line in deque(f, maxlen = 10000000):
      if(findStr in line):
        daemon+= [line.strip()]

  return daemon

In [32]:
daemonlog= search("Hammered/daemon.log", 'WARNING')
daemonlog

['Mar 18 10:18:42 app-1 /etc/mysql/debian-start[7566]: WARNING: mysql.user contains 2 root accounts without password!',
 'Mar 18 17:01:44 app-1 /etc/mysql/debian-start[14717]: WARNING: mysql.user contains 2 root accounts without password!',
 'Mar 22 13:49:49 app-1 /etc/mysql/debian-start[5599]: WARNING: mysql.user contains 2 root accounts without password!',
 'Mar 22 18:43:41 app-1 /etc/mysql/debian-start[4755]: WARNING: mysql.user contains 2 root accounts without password!',
 'Mar 22 18:45:25 app-1 /etc/mysql/debian-start[4749]: WARNING: mysql.user contains 2 root accounts without password!',
 'Mar 25 11:56:53 app-1 /etc/mysql/debian-start[4848]: WARNING: mysql.user contains 2 root accounts without password!',
 'Apr 14 14:44:34 app-1 /etc/mysql/debian-start[5369]: WARNING: mysql.user contains 2 root accounts without password!',
 'Apr 14 14:44:36 app-1 /etc/mysql/debian-start[5624]: WARNING: mysqlcheck has found corrupt tables',
 'Apr 18 18:04:00 app-1 /etc/mysql/debian-start[4647]: WA

In [43]:
auth= search("Hammered/auth.log", 'USER=root')
auth

['Mar 16 08:12:09 app-1 sudo:     user3 : TTY=tty1 ; PWD=/home/user3 ; USER=root ; COMMAND=/bin/su',
 'Mar 16 08:27:37 app-1 sudo:     user3 : TTY=pts/0 ; PWD=/home/user3 ; USER=root ; COMMAND=/bin/su',
 'Mar 16 10:14:10 app-1 sudo:     user3 : TTY=pts/1 ; PWD=/home/user3 ; USER=root ; COMMAND=/bin/su',
 'Mar 16 17:12:41 app-1 sudo:     user3 : TTY=pts/0 ; PWD=/opt/software ; USER=root ; COMMAND=/bin/su',
 'Mar 16 17:13:52 app-1 sudo:     user3 : TTY=pts/0 ; PWD=/opt/software ; USER=root ; COMMAND=/bin/su',
 'Mar 18 09:43:06 app-1 sudo:     user3 : TTY=tty1 ; PWD=/home/user3 ; USER=root ; COMMAND=/bin/su',
 'Mar 18 09:49:52 app-1 sudo:     user3 : TTY=tty1 ; PWD=/opt/software ; USER=root ; COMMAND=/bin/su',
 'Mar 18 09:51:23 app-1 sudo:     user3 : TTY=tty1 ; PWD=/home/user3 ; USER=root ; COMMAND=/bin/su',
 'Mar 18 09:56:22 app-1 sudo:     user3 : TTY=tty1 ; PWD=/home/user3 ; USER=root ; COMMAND=/bin/su',
 'Mar 18 10:00:36 app-1 sudo:     user3 : TTY=pts/1 ; PWD=/home/user3 ; USER=root

In [44]:
!pip install apachelogs

from collections import deque, Counter
import re
import apachelogs
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
pd.set_option("display.max_rows", None)

In [46]:
parser = apachelogs.LogParser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")

In [52]:
accesslog= search("Hammered/apache2/www-access.log", '')
accesslog[0:5]

['10.0.1.2 - - [19/Apr/2010:06:36:15 -0700] "GET /feed/ HTTP/1.1" 200 16605 "-" "Apple-PubSub/65.12.1" C4nE4goAAQ4AAEP1Dh8AAAAA 3822005',
 '10.0.1.2 - - [19/Apr/2010:07:05:25 -0700] "GET /feed/ HTTP/1.1" 200 16605 "-" "Apple-PubSub/65.12.1" c@FurwoAAQ4AAEP2EQYAAAAC 3043815',
 '10.0.1.2 - - [19/Apr/2010:07:33:47 -0700] "GET /feed/ HTTP/1.1" 200 16605 "-" "Apple-PubSub/65.12.1" 2UiswgoAAQ4AAEP3EbIAAAAD 4687677',
 '10.0.1.2 - - [19/Apr/2010:08:02:10 -0700] "GET /feed/ HTTP/1.1" 200 16605 "-" "Apple-PubSub/65.12.1" PsoODgoAAQ4AAEP4ElQAAAAE 717069',
 '123.4.59.174 - - [19/Apr/2010:08:26:30 -0700] "GET http://proxyjudge1.proxyfire.net/fastenv HTTP/1.1" 404 1466 "-" "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)" ldqydQoAAQ4AAEP5EvsAAAAF 2358754']

In [60]:
log_line = accesslog
df_col = ["ip", 'time', "request", "status", "size"]
df_row = []
for x in log_line:
  p1 = parser.parse(x.replace(re.findall('( [a-zA-Z0-9@\-]+ [0-9]+)', x)[-1], ""))
  df_row += [[p1.remote_host,p1.request_time.strftime("%m/%d/%Y, %H:%M:%S"),
             p1.request_line, p1.final_status, p1.bytes_sent]]

df = pd.DataFrame(data=df_row, columns=df_col)
  

In [64]:
df.head()

,ip,time,request,status,size
0,10.0.1.2,"04/19/2010, 06:36:15",GET /feed/ HTTP/1.1,200,16605.0
1,10.0.1.2,"04/19/2010, 07:05:25",GET /feed/ HTTP/1.1,200,16605.0
2,10.0.1.2,"04/19/2010, 07:33:47",GET /feed/ HTTP/1.1,200,16605.0
3,10.0.1.2,"04/19/2010, 08:02:10",GET /feed/ HTTP/1.1,200,16605.0
4,123.4.59.174,"04/19/2010, 08:26:30",GET http://proxyjudge1.proxyfire.net/fastenv H...,404,1466.0


In [71]:
df.nunique()

ip          27
time       326
request     16
status       7
size        13
dtype: int64

Answers to questions

In [ ]:
#1. Which service did the attackers use to gain access to the system?


In [ ]:
#2. What is the operating system version of the targeted system? (one word)



In [ ]:
#3. What is the name of the compromised account?



In [ ]:
#4. Consider that each unique IP represents a different attacker. How many attackers were able to get access to the system?

In [ ]:
#5. Which attacker's IP address successfully logged into the system the most number of times?



In [ ]:
#6. How many requests were sent to the Apache Server?



In [ ]:
#7. How many rules have been added to the firewall?



In [ ]:
#8. One of the downloaded files to the target system is a scanning tool. Provide the tool name.



In [ ]:
#9. When was the last login from the attacker with IP 219.150.161.20? Format: MM/DD/YYYY HH:MM:SS AM





In [ ]:
#10. The database displayed two warning messages, provide the most important and dangerous one.



In [ ]:
#11. Multiple accounts were created on the target system. Which one was created on Apr 26 04:43:15?



In [ ]:
#12. Few attackers were using a proxy to run their scans. What is the corresponding user-agent used by this proxy?

